# 3D reconstruction using 2 image

Key References
- https://www.opencvhelp.org/tutorials/advanced/reconstruction-opencv/

In [ ]:
from __future__ import annotations

from typing import TYPE_CHECKING

import cv2 as cv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pyrender
import scipy.spatial.transform as sci_trans

import project_3d_reconstruction.mpl as proj_mpl
from project_3d_reconstruction.rendering import render_helper as rh

if TYPE_CHECKING:
    from mpl_toolkits.mplot3d.axes3d import Axes3D

# Generate simple scene

In [ ]:
"""
Attempting to recreate the matplotlib one
"""

renderer = rh.RenderHelper()
renderer.addCube(0.5, rh.positionOnly(-0.5, -0.5, -0.5), color=[255, 0, 0])
renderer.addCube(0.4, rh.positionOnly(-0.4, 0.35, -0.6), color=[144, 144, 0])
renderer.addCube(0.4, rh.positionOnly(-0.8, 0.35, -0.6), color=[144, 0, 144])
renderer.addCube(0.4, rh.positionOnly(0.5, -0.35, 0.6), color=[0, 100, 144])
renderer.addCube(0.2, rh.positionOnly(0.2, -0.43, -0.27), color=[144, 0, 144])
renderer.addCube(0.2, rh.positionOnly(-0.263, 0.375, 0.03), color=[0, 0, 144])

for i in [-0.2, 0, 0.2]:
    renderer.addCube(0.3, rh.positionOnly(i, i, i), color=[50 + 200 * i, 255, 0])

renderer.addCube(0.5, rh.positionOnly(0.5, 0.5, 0.5), color=[0, 0, 255])

# TODO add more surface feature points/texture

# Save image from two perspectives.

In [ ]:
# Render two views
pose1 = rh.pointingAtOrigin(-0.2, 2)
renderer.moveCamera(pose1)
renderer.render(show_image=False, image_filename="test1.png")

pose2 = rh.pointingAtOrigin(0.2, 2)
renderer.moveCamera(pose2)
renderer.render(show_image=False, image_filename="test2.png")

image_1 = cv.imread("test1.png")
image_2 = cv.imread("test2.png")

# actual extrinsics
print(pose1)
print(pose2)
relativePose = np.matmul(np.linalg.inv(pose2), pose1)
print(relativePose)
# camCopy = pyrender.IntrinsicsCamera(fx=512, fy=512, cx=256, cy=256)
# print(camCopy.get_projection_matrix(width=512,height=512))

gray_1 = cv.cvtColor(image_1, cv.COLOR_BGR2GRAY)
gray_2 = cv.cvtColor(image_2, cv.COLOR_BGR2GRAY)

ifig, iaxs = plt.subplots(ncols=2, figsize=(8, 8))
iaxs[0].imshow(image_1)
iaxs[1].imshow(image_2)
ifig.tight_layout()

[[1, 0, 0, -0.2], [0, 0.7071067811865476, -0.7071067811865476, -2], [0, 0.7071067811865476, 0.7071067811865476, 2], [0, 0, 0, 1]]
[[1, 0, 0, 0.2], [0, 0.7071067811865476, -0.7071067811865476, -2], [0, 0.7071067811865476, 0.7071067811865476, 2], [0, 0, 0, 1]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -4.00000000e-01]
 [ 0.00000000e+00  1.00000000e+00 -1.01465364e-17  0.00000000e+00]
 [ 0.00000000e+00 -1.01465364e-17  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


# Detect SIFT features

In [ ]:
def opponentSIFT(img):
    # Step 1 convert to opponent color space
    # TODO optimize
    B = img[:, :, 0]
    G = img[:, :, 1]
    R = img[:, :, 2]

    O1 = np.divide((R - G), np.sqrt(2))
    O2 = np.divide((R + G - 2 * B), np.sqrt(6))
    O3 = np.divide((R + G + B), np.sqrt(3))
    # visually check opponent color space
    # cv.imwrite('sift_keypointsO1.jpg',np.uint8(O1))
    # cv.imwrite('sift_keypointsO2.jpg',np.uint8(O2))
    # cv.imwrite('sift_keypointsO3.jpg',np.uint8(O3))

    # Step 2 use Harris-Laplace point detector on intensity channel (o3)
    # TODO use a real point detector or figure out what parameters to use with cv SIFT
    # use this space to specify additional parameters
    sift = cv.SIFT_create()
    # sift = cv.SIFT_create(nfeatures=1000, nOctaveLayers=3, sigma=10)

    kp = sift.detect(np.uint8(O3), None)

    # Step 3 compute descriptors for each opponent channel
    _, des1 = sift.compute(np.uint8(O1), kp)
    _, des2 = sift.compute(np.uint8(O2), kp)
    _, des3 = sift.compute(np.uint8(O3), kp)

    # combine into one large descriptor
    des = np.concatenate((des1, des2, des3), axis=1)

    return kp, des


kp1, des1 = opponentSIFT(image_1)
kp2, des2 = opponentSIFT(image_2)

image_kp_1 = cv.drawKeypoints(
    gray_1, kp1, image_1, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
image_kp_2 = cv.drawKeypoints(
    gray_2, kp2, image_2, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

ifig, iaxs = plt.subplots(ncols=2, figsize=(8, 8))
iaxs[0].imshow(image_kp_1)
iaxs[1].imshow(image_kp_2)

# Match features (brute-force)

In [ ]:
bf = cv.BFMatcher()
bf_matches = bf.knnMatch(des1, des2, k=2)
good = []

for m, n in bf_matches:
    if m.distance < 0.8 * n.distance:
        good.append([m])

image_matches = cv.drawMatchesKnn(
    image_1,
    kp1,
    image_2,
    kp2,
    good,
    None,
    flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)


ifig, iax = plt.subplots()
iax.imshow(image_matches)
ifig.tight_layout()

# Estimate essential matrix

In [ ]:
# points_1 = np.array([kp1[match.queryIdx].pt for [match] in good])[:, np.newaxis, :]
# points_2 = np.array([kp2[match.trainIdx].pt for [match] in good])[:, np.newaxis, :]
points_1 = np.int32([kp1[m.queryIdx].pt for [m] in good]).reshape(-1, 1, 2)
points_2 = np.int32([kp2[m.trainIdx].pt for [m] in good]).reshape(-1, 1, 2)

# see pyrender script for intrinsic camera params
intrinsic_mat = np.array([[512, 0, 256], [0, 512, 256], [0, 0, 1]])
essential_mat, mask = cv.findEssentialMat(
    points_1, points_2, intrinsic_mat, method=cv.RANSAC, prob=1 - 1e-12, threshold=3
)
_, est_rot, est_trans, _ = cv.recoverPose(
    essential_mat, points_1, points_2, intrinsic_mat, mask=mask
)

# check the matches kept by ransac - sometimes they're bad
postRansac = []
for i in range(len(mask)):
    if mask[i]:
        postRansac.append(good[i])
print(len(good))
print(len(postRansac))
image_matches = cv.drawMatchesKnn(
    image_1,
    kp1,
    image_2,
    kp2,
    postRansac,
    None,
    flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)

ifig, iax = plt.subplots()
iax.imshow(image_matches)
ifig.tight_layout()

34
32


In [ ]:
# check estimated extrinsic params
print(est_rot)
print(est_trans)

[[ 0.99978913  0.01512054  0.01389451]
 [-0.01508983  0.99988347 -0.00231218]
 [-0.01392785  0.00210203  0.99990079]]
[[-0.99996851]
 [ 0.00652723]
 [ 0.00451478]]


In [ ]:
est_extrinsic = np.hstack((est_rot, est_trans))

# act_rot = sci_trans.Rotation.align_vectors(dir_2, dir_1)[0].as_matrix()
# act_extrinsic = np.hstack((act_rot, np.reshape(dir_2 - dir_1, (-1, 1))))
act_extrinsic = relativePose[0:3, :]
print(est_extrinsic)
print(act_extrinsic)

origin = np.hstack((np.eye(3), np.zeros((3, 1))))

all_extrinsics = [origin, act_extrinsic, est_extrinsic]
base_colors = [
    mpl.colors.to_rgb("tab:blue"),
    mpl.colors.to_rgb("tab:green"),
    mpl.colors.to_rgb("tab:red"),
]
cfig, cax = plt.subplots(subplot_kw={"projection": "3d"})
for mat, color in zip(all_extrinsics, base_colors, strict=False):
    pos = mat[:, 3]
    for column, weight in zip(mat[:, :3].T, [1, 0.8, 0.5], strict=False):
        cax.quiver(*pos, *column, color=np.array(color) * weight)

cfig.tight_layout()
cax.set_xlim3d(-1, 1)
cax.set_ylim3d(-1, 1)
cax.set_zlim3d(-1, 1)
# cax.voxels(voxelarray, facecolors=colors, edgecolors=colors);

[[ 0.99978913  0.01512054  0.01389451 -0.99996851]
 [-0.01508983  0.99988347 -0.00231218  0.00652723]
 [-0.01392785  0.00210203  0.99990079  0.00451478]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -4.00000000e-01]
 [ 0.00000000e+00  1.00000000e+00 -1.01465364e-17  0.00000000e+00]
 [ 0.00000000e+00 -1.01465364e-17  1.00000000e+00  0.00000000e+00]]


(-1.0, 1.0)